In [2]:
import os
import cv2
import base64

def extract_frames(video_path, output_folder, interval=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_frames = []

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        if frame_count % interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
            extracted_frames.append(frame_filename)

        frame_count += 1

    video_capture.release()
    return extracted_frames

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def query_model(video_path, prompt):
    frames = extract_frames(video_path, output_folder)
    batch_size = 8
    responses = []
    for i in range(0, len(frames), batch_size):
        batch = frames[i:i + batch_size]

        contents = [{
            "type": "text",
            "text": prompt,
        }]

        for frame_path in batch:
            encoded_image = encode_image_to_base64(frame_path)
            contents.append({
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{encoded_image}"
            })

        messages = [
            {
                "role": "user",
                "content": contents
            }
        ]

        response = client.chat.complete(model=model, messages=messages)
        responses.append(response.choices[0].message.content)

        summary_prompt = "Summarize the following into one concise paragraph:\n\n" + " ".join(responses)
        summary_response = client.chat.complete(model=model, messages=[{"role": "user", "content": [{"type": "text", "text": summary_prompt}]}])

    return summary_response.choices[0].message.content  # Merge all responses

In [ ]:
from mistralai import Mistral

MISTRAL_API_KEY =  "REDACTED"
model = "pixtral-12b-2409"
output_folder = 'extracted_frames'

client = Mistral(api_key=MISTRAL_API_KEY)
model = "pixtral-12b-2409"

video_path = 'resources/Fire Extinguisher Safety - AFIRE100.mp4'
prompt = '''Summarize what this video is about including skills you'll learn etc.'''

response = query_model(video_path, prompt)
response

'The series of images depicts a user navigating Delta\'s learning management system to access and complete an online course titled "Fire Extinguisher Safety." The course, identified by the code AFRE100, educates users on the proper use and safety measures related to fire extinguishers. Key sections include an introduction, navigation guide, objectives, different types of fire extinguishers, and specific guidelines for Delta and Delta Connection Carriers. Users learn how to identify various types of fire extinguishers, use them correctly, and respond to fires, particularly in ramp and aircraft scenarios. The course includes interactive elements such as quizzes and visual aids to enhance learning and ensure comprehension, culminating in an assessment to verify understanding.'